In [7]:
# Import the required modules
import visa
import time
import numpy as np
from scipy.optimize import minimize
import matplotlib.pyplot as plt
from Keithley import Agilent_81635A
from Agilent_81635A import Agilent_81635A
from Agilent_E3646A import Agilent_E3646A

In [101]:
# Coordinates descent Algorithm 
def sweep_bias(MRF,channel):
    """Coarse tuning of a MRF object using the coordinates descent algorithm."""
    
    # Possible bias values
    bias_min = 0
    bias_max = 4
    bias_points = 100
    bias_testpoints = np.linspace(bias_min,bias_max,bias_points).tolist()
    
    # Turn on the laser and set the wavelength
    MRF.PD.set_wavelength(1550)
    MRF.PD.laser_on()
   
    power_list = []
    for k in bias_testpoints: # For each bias value
        MRF.apply_bias(channel,k)
        #time.sleep(1)
        power_list.append(MRF.PD.measure_power())
    plotsweep(bias_testpoints, power_list)
    MRF.apply_bias(channel,bias_testpoints[power_list.index(min(power_list))])
    # Turn on the laser 
    MRF.PD.laser_off()

# Coordinates descent Algorithm 
def CoordsDescent(MRF,number_iter):
    """Coarse tuning of a MRF object using the coordinates descent algorithm."""
    
    # Possible bias values
    bias_min = 0
    bias_max = 4
    bias_points = 100
    bias_testpoints = np.linspace(bias_min,bias_max,bias_points).tolist()
    
    # Turn on the laser and set the wavelength
    MRF.PD.laser_on()
    for i in range(1,number_iter+1): # For each iteration
        for j in range(0,MRF.num_parameters): # For each ring
            power_list = []
            for k in bias_testpoints: # For each bias value
                MRF.apply_bias(j+1,k)
                #time.sleep(1)
                power_list.append(MRF.PD.measure_power())
            plotsweep(bias_testpoints, power_list)
            MRF.apply_bias(j+1,bias_testpoints[power_list.index(min(power_list))])
    # Turn on the laser 
    MRF.PD.laser_off()
            
# Nelder Mead simplex algorithm
def NelderMead(MRF):
    """Fine tuning of a MRF object using the Nelder Mead simplex algorithm"""
    # Turn on the source
    MRF.PD.laser_on()
    # Initial guess
    x0 = MRF.applied_bias
    # Optimization
    res = minimize(MRF.minimize_MRF, x0, method='Nelder-Mead', tol=1e-6, options={'disp': True})
    #Turn off the source
    MRF.PD.laser_off()
    return res.x

def tuneMRF(MRF, wavelength):
    """Tune/stabilise the MRF object using coarse + fine algorithms"""
   
    MRF.PD.set_wavelength(wavelength)
    
    CoordsDescent(MRF,2) # Coordinates descent
    NelderMead(MRF) # Nelder Mead
    
def plotsweep(bias, power):
    plt.plot(bias, power)
    plt.plot([bias[power.index(min(power))]], [min(power)], marker='o', markersize=10, color="red")
    plt.xlabel("Bias [V]")
    plt.ylabel("Power [dBm]")
    plt.show()

In [102]:
class RealMRF(object):
    """"""
    
    def __init__(self,instruments):
        self.PD = instruments['PD']      
        self.DCsources = instruments['DCsources']
        self.num_parameters = len(self.DCsources)
        self.applied_bias = [0.] * self.num_parameters
    
    def connect_instruments(self):
        """Conncet the DC sources remotely."""
        # Connect the power sensor
        self.PD.connect()
        # Connect the DC sources
        for instrument in self.DCsources:
            instrument.connect()
    
    def apply_bias(self,source_num,bias):
        """Set the bias for the ring #[ring_number] at [bias_value]"""
        self.applied_bias[source_num-1] = bias
        self.DCsources[source_num-1].source_voltage(bias)
        
    def DC_off(self):
        """Turn off the DC bias for all DC sources."""
        for i in range(self.num_parameters):
            self.apply_bias(i,0)      
    
    def test_MRF(self,bias_list):
        """"""
        # Apply bias to DC sources
        for i in range(self.num_parameters):
            self.apply_bias(i+1,bias_list[i])
        # Measure the optical power on the sensor
        return float(self.PD.measure_power())
    
    def minimize_MRF(self,bias_list):
        """Minimization optimisation function for the MRF object"""
        return self.test_MRF(bias_list)

In [122]:
instruments = {'PD': Agilent_81635A(2,1),
             'DCsources': [Keithley_2612B('a'), Keithley_2612B('b'), Agilent_E3646A('1')]}

mrf = RealMRF(instruments)
#print mrf.num_parameters
mrf.connect_instruments()
#mrf.apply_bias(3,0)
#print mrf.minimize_MRF([0,0])

In [123]:
mrf.DC_off()
tuneMRF(mrf,1550)

In [ ]:
mrf.apply_bias(3,2)

In [124]:
mrf.DC_off()

In [ ]:
PD.laser_off()

In [ ]:
mrf.DC_off()
sweep_bias(mrf,3)